In [31]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np

In [32]:
# project_name = 'Split_Sys_VQA'
project_name = 'Split_Sub_VQA'
# exp_name = 'nc-Naive'       # nc-
exp_name = 'nc-ER'
# exp_name = 'nc-LwF'
# exp_name = 'GEM'
test_case_name = 'Novel-{}'.format(exp_name)
# test_case_name = 'Novel-{}-frz'.format(exp_name)
# test_case_name = 'Novel-{}-lm'.format(exp_name)
# test_case_name = 'Novel-{}-frz-lm'.format(exp_name)

In [33]:
results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-{test_case_name}.npy'), allow_pickle=True)
print(results)

[{'Top1_Acc_Epoch/train_phase/train_stream/Task000': 1.0, 'Loss_Epoch/train_phase/train_stream/Task000': 0.002793627093409414, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.92, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.26589810848236084, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.92, 'Loss_Stream/eval_phase/test_stream/Task000': 0.26589810848236084}
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 1.0, 'Loss_Epoch/train_phase/train_stream/Task000': 0.00368402004547295, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.92, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 0.26589810848236084, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.95, 'Loss_Stream/eval_phase/test_stream/Task000': 0.16256767511367798, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.95, 'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 0.16256767511367798}
 {'Top1_Acc_Epoch/train_phase/train_stream/Task000': 1.0, 'Loss_Epoch/train_phase/train_stream/Task000': 0.0

In [34]:
accs = [result['Top1_Acc_Stream/eval_phase/test_stream/Task000'] for result in results]
print(accs)

[0.92, 0.95, 0.9, 0.96, 0.97, 0.91, 0.99, 0.94, 0.71, 0.95]


In [35]:
print(f'{np.mean(accs)*100:.2f}% ({np.std(accs)*100:.2f}%)')

92.00% (7.47%)
